In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Shivansh Jayara\\MyWork\\MLOps\\chicken_disease\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'c:\\Users\\Shivansh Jayara\\MyWork\\MLOps\\chicken_disease'

In [46]:
from dataclasses import dataclass
from pathlib import Path

"""
for entity: config entity
"""
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [47]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [70]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
            This will return all the configuration for data ingestion. Like:
            - root folder:  This is a root folder named 'Artifacts/Ingestion/'
            - source url: This the url from where you can get the data set in zip format
            - local data file: This the path where that zip data set is saved
            - unzip directory: This is the path where that data set is unzip and saved
        """
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
                                                    root_dir=config.root_dir,
                                                    source_URL=config.source_URL,
                                                    local_data_file=config.local_data_file,
                                                    unzip_dir=config.unzip_dir 
                                                )
        return data_ingestion_config      

In [71]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [72]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [74]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-29 15:38:58,419: INFO: common: yaml file: config\config.yaml loaded successfully]
reading config file
[2023-06-29 15:38:58,423: INFO: common: yaml file: params.yaml loaded successfully]
reading params file
[2023-06-29 15:38:58,428: INFO: common: created directory at: artifacts]
[2023-06-29 15:38:58,429: INFO: common: created directory at: artifacts/data_ingestion]
[2023-06-29 15:39:02,771: INFO: 1696126359: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7D9C:6B4B:75567:AFC46:649D583C
Accept-Ranges: bytes
Date: Thu, 29 Jun 2023 10:09:01 GMT
Via: 1.1 varnish
X-Ser